In [1]:
import time
import boto3

In [2]:
ACCESS_KEY = ""
SECRET_KEY = ""

In [4]:
def transcribe_file(job_name, file_uri, transcribe_client):
    transcribe_client.start_transcription_job(
        TranscriptionJobName=job_name,
        Media={'MediaFileUri': file_uri},
        MediaFormat='mp4',
        LanguageCode='en-US'
    )

    max_tries = 60
    while max_tries > 0:
        max_tries -= 1
        job = transcribe_client.get_transcription_job(TranscriptionJobName=job_name)
        job_status = job['TranscriptionJob']['TranscriptionJobStatus']
        if job_status in ['COMPLETED', 'FAILED']:
            print(f"Job {job_name} is {job_status}.")
            if job_status == 'COMPLETED':
                print(
                    f"Download the transcript from\n"
                    f"\t{job['TranscriptionJob']['Transcript']['TranscriptFileUri']}.")
                return job['TranscriptionJob']['Transcript']
            #break
        else:
            print(f"Waiting for {job_name}. Current status is {job_status}.")
        time.sleep(10)

In [5]:
transcribe_client = boto3.client('transcribe',
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY, region_name='us-east-2')
#    aws_session_token=SESSION_TOKEN)
file_uri = 's3://test-jean/query.mp4'
res = transcribe_file('test-query', file_uri, transcribe_client) 

ConflictException: An error occurred (ConflictException) when calling the StartTranscriptionJob operation: The requested job name already exists. Use a different job name.

In [33]:
result = transcribe_client.get_transcription_job(
    TranscriptionJobName='test-query'
) 

In [37]:
import json
import requests
res = requests.get(result['TranscriptionJob']['Transcript']['TranscriptFileUri']).text

In [50]:
query = dict(json.loads(res))['results']['transcripts'][0]['transcript']

'get first name in customer.'

In [52]:
# ML SQL ValueNet

0

In [54]:
os.popen('curl -F "csv=@bridges.csv" -F "q=how long is throgs neck" localhost:5050').read()

'{"answer":[1800],"params":["throgs neck"],"sql":"SELECT (length) FROM bridges WHERE bridge = ?"}\n'

In [8]:
# Using above results, send to Polly
polly_client = boto3.Session(
                aws_access_key_id=ACCESS_KEY,                     
    aws_secret_access_key=SECRET_KEY,
    region_name='us-east-2').client('polly')

response = polly_client.synthesize_speech(VoiceId='Joanna',
                OutputFormat='mp3', 
                Text = result)

file = open('speech.mp3', 'wb')
file.write(response['AudioStream'].read())
file.close()

In [ ]:
transcribe_client = boto3.client('transcribe',
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY, region_name='us-east-2')
#    aws_session_token=SESSION_TOKEN)
file_uri = 's3://test-jean/query.mp4'
res = transcribe_file('test-query', file_uri, transcribe_client) 